# Kickstarting the Martian Greenhouse Effect
## A Final Project for Physics 361 by Morgan Baxter and Alex Brovender

For a variety of reasons, including proximity to the sun and atmospheric and regolith composition, Mars is the number one candidate body in the solar system for future human colonization. Unfortunately, it is currently highly inhospitable to human life, partially due to the extreme surface temperatures that can range from $-143^{\circ} C$ to $35^{\circ} C$. We use simple climate models to find the amount of energy that would be required to liberate enough CO2 to kickstart a greenhouse effect and warm the planet.

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/6/6a/PIA22546-Mars-AnnualCO2ice-N%26SPoles-20180806.gif" width="500"/>
</div>
$\textbf{Figure 1:}$ Extents of north (left) and south (right) polar CO2 ice during a martian year

$\textbf{Source:}$ NASA/JPL-caltech




## Hogg 2008: Glacial cycles and carbon dioxide: A conceptual model

We utilize Hogg, 2008, as a starting point for our model for the Mars climate system. The equations as he provides them are as follows:

$$ c\frac{dT}{dt} = \bar{S} + \sum_i{sin \left (\frac{2 \pi t}{\Gamma_i}\right )} + \bar{G} + A \ln \left(\frac{C}{C_0}\right) - \sigma T^4$$

$$ \frac{dC}{dt} = V - (W_0 + W_1C) + \beta (C_{max} - C)\max\left(\frac{dT}{dt} - \epsilon , 0 \right) $$

These equations in their current form are not useful to us for a variety of reasons. They include terms that account for volcanic activity, the carbon sink of the oceans, orbital pertubations, and the heat capacity of the oceans. As a result, the coupled differential equations that we will use look slightly different. However, they do take a variety of the the phenomena we want to observe into account, so they are a good starting point.

$\sum_i{sin \left (\frac{2 \pi t}{\Gamma_i}\right )}$ gives us variations of incoming solar radiation due to orbit. Should be neglegible.

c = specific heat capacity of ocean

$\frac{dT}{dt}$ = Temp/time derivative on Mars's surface?

$ C(t) $ = atmospheric concentration of $CO_2$

$ C_0 = 280 $ ppm initial concentration

$ A = 5.35 W/m^2 $ effect of $CO_2$ on radiation budget

$V$ constant source of $CO_2$ from volcanos. We set this to zero.

$W_0 = 0.013 $ ppm/year, $W_1 = \frac{1}{12000}$ years, these values relate to carbon sequestering by rocks and the Ocean

$\bar{S}$ = Average Incoming Solar Radiation on Mars, units of J?/year

$\beta = 0.38 ^\circ C^{-1}  $ scaling parameter 

$\epsilon$ dependent on timescale of orbital forcing

$V = C0_2$ source from volcanoes(0.018-0.03 ppm/yr), we aren't taking this term into account

Here, $F_S$ is the solar constant, in units of $W m^{-2}$, and is a different value for each body in the solar system. The solar constant for Mars is approximately $589 W m^{-2}$. $A$ is the planetary bond albedo, which is dimensionless and equal to 0.25. $\sigma$ is the Stefan-Boltzmann constant, which is equal to $\frac{2 \pi^5 k^4}{15c^2 h^3} = 5.67 *10^{-8} W m^{-2} K^{-4}$.

## Modified Equations

Our reduced equations look as follows:

$$ \frac{dT}{dt} = \Big[ \bar{S} + \bar{G} + A \ln \left(\frac{C}{C_0}\right) - \sigma T^4 \Big] /c$$

$$\frac{dC}{dt} = (T^* - 120)\beta$$

Our state variables are $T$, the average surface temperature in Kelvin, and $C$, the mass of CO2 in the Martian atmosphere, in kilograms. Therefore, our parameters must match these units, and we list them below:

$\beta$ is the rate of carbon effect on temperature. It has units of $kg \cdot s^{-1} K^{-1}$.

$T^*$ is the temperature $\textbf{at the poles}$ in Kelvin. This will vary by a set offset $\textbf{NOT YET DETERMINED}$ that will solve issues relating to the average temperature on Mars being more than warm enough to sublimate CO2 at the given pressure.

$\bar{S}$ is the rate of incoming solar radiation. This value is $589 W \cdot m^{-2}$

$\bar{G}$ is the greenhouse warming term. This should initially be on the order of $1 W \cdot m^{-2}$

$A$ represents the effect of CO2 on the radiation budget. It should be set to $5.35 W \cdot m^{-2}$. Might want to play around for this value. Hogg cites a paper that seems to detail where this value comes from.

$\frac{C}{C_0}$ represents the ratio between the mass of the CO2 currently in the atmosphere compared to the initial value. It is dimensionless.

$c$ is the specific heat capacity of CO2. We will take this to be approximately $700 J \cdot kg^{-1} \cdot K^{-1}$. This is a function of temperature and pressure, and therefore it could be more accurate to have this in function form rather than as a constant.

These two equations are enough for us to solve the system.

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integ

# this function defines the system of equations that we wrote above.
def RHS(w,t,p):
        
    # w holds C and T, the mass of atmospheric carbon and average surface temperature
    C, T = w
    
    # the variable p holds the parameters
    
    SB_constant, solar_constant, G, A, C_0, beta = p
    
    dT = (solar_constant + G + A*np.ln(C/C_0) - (SB_constant*(T**4)))/700
    
    dC = (T - 120)*beta
    
    f = [dT, dC]
    
    return f

In [ ]:
#solve = integ.solve_ivp(RHS,,[0.95])


# here we define our constants and put them in a list to hand to the RHS function
SB_constant = 5.67e-8
solar_constant = 589
G = 1
a = 5.35
C_0 = 
beta = 0.1
parameters = [SB_constant, solar_constant, G, A, C_0, beta]